# Tictactrip : Crunching de data

### Objectifs

- Extraire des infos intéressantes type :
    - prix min, moyen et max
    - durée min, max, moyenne par trajet


- Différence de prix moyen et durée selon le train, le bus et le covoit selon la distance du trajet
    
    Par exemple (0-200km, 201-800km, 800-2000km, 2000+km)
    
- Le plus d’infos bonus !
    
    Comme par exemple : *Graphes, prédictions de prix, rapport des soucis relevés dans les données, visualisation interactive, sourcing & utilisation de données externes pertinentes, utilisation d’API externes* 


### Méthodologie

- Chargement et compréhension des Données
- Extraction d'informations sur les données
- Comparaisons (suivant le prix et durée) des méthodes de transport par tranche de distance
- Bonus 
- Ressources

<div style="background-color:#5bbbcf;padding:2px;border-radius:4px">  
    <h1 align="center" style="color:white;">0. Chargement et compréhension des données</h1>  
</div>

In [1]:
# Importation des bibliothèques
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

cmr = sns.light_palette("red", as_cmap=True)

### Chargement des données

In [2]:
cities = pd.read_csv('/kaggle/input/tictactrip/cities.csv')
providers = pd.read_csv('/kaggle/input/tictactrip/providers.csv')
stations = pd.read_csv('/kaggle/input/tictactrip/stations.csv')
ticket_data = pd.read_csv('/kaggle/input/tictactrip/ticket_data.csv')

cities.shape, providers.shape, stations.shape, ticket_data.shape

((8040, 6), (227, 10), (11035, 4), (74168, 12))

### Compréhension des données

#### Villes

Le dataset `cities` contient des informations sur les villes desservies par tictactrip. On peut y trouver des informations comme latitude, longitude, nom, population de chaque ville.

Nous disposons de **8040** instances de données dans ce dataset.

In [3]:
print(f"Instances de données disponibles : {cities.shape[0]} \n")
cities.head()

Instances de données disponibles : 8040 



,id,local_name,unique_name,latitude,longitude,population
0,5159,"Padua, Veneto, Italia",padua,45.406435,11.876761,209678.0
1,76,"Barcelona, Cataluña, España",barcelona,41.385064,2.173404,1611822.0
2,81,"Basel, Basel-Stadt, Schweiz",basel,47.593437,7.619812,NaN
3,259,"Erlangen, Bayern, Deutschland",erlangen,49.589674,11.011961,105412.0
4,11979,"Balș, Olt, România",balș,44.353354,24.095672,NaN


In [4]:
cities.query("id == 542")

,id,local_name,unique_name,latitude,longitude,population
2630,542,"Montpellier, Occitanie, France",montpellier,43.604452,3.918318,275318.0


#### Fournisseurs (Prestataires)

Le dataset `providers` contient des informations sur les différents fournisseurs. Un fournisseurs est une "sous-compagnie". On peut y trouver des informations comme le nom des fournisseurs, les identifiants des compagnies, les moyens de transport que propose chaque compagnie, les caractéristiques des moyens de transport, etc...

Il y a **227** instances de données disponibles dans ce dataset.

In [5]:
print(f"Instances de données disponibles : {providers.shape[0]} \n")
providers.head()

Instances de données disponibles : 227 



,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
0,9,1,NaN,ouibus,Ouibus,True,True,True,False,bus
1,10,2,NaN,deinbus,Deinbus.de,False,False,False,False,bus
2,11,3,NaN,infobus,Infobus,False,False,False,False,bus
3,12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus
4,13,5,NaN,flixbus,Flixbus,True,False,False,False,bus


In [6]:
# Différents moyens de transport disponible
providers.transport_type.value_counts()

bus           214
train          10
carpooling      2
car             1
Name: transport_type, dtype: int64

Nous remarquons dans ce dataset que les 227 différents moyens de transport sont réparti de manière suivante : 

- 214 **bus**;
- 10 **train**;
- 2 **carpooling** (covoiturage);
- 1 **car** (voiture).

#### Stations

Le dataset `stations` contient des informations sur les stations desservies par tictactrip. On peut y trouver des informations comme latitude, longitude, nom de chaque station. 

Nous avons la présence de **11035** instances de données dans ce dataset.

In [7]:
print(f"Instances de données disponibles : {stations.shape[0]} \n")
stations.head()

Instances de données disponibles : 11035 



,id,unique_name,latitude,longitude
0,1,Aalen (Stuttgarter Straße),48.835296,10.092956
1,2,Aéroport Bordeaux-Mérignac,44.830226,-0.700883
2,3,Aéroport CDG,49.009900,2.559310
3,4,Aéroport de Berlin-Schönefeld,52.389446,13.520345
4,5,Aéroport de Dresden,51.123604,13.764737


#### Tickets

Le dataset `ticket_data` contient des informations sur les tickets desservies par tictactrip. On peut y trouver des informations comme les trajets, les identifiants des villes de départ et d'arrivé, le prix de chaque trajet, etc... 

Nous avons **74168** instances de données disponibles dans ce dataset.

In [8]:
print(f"Instances de données disponibles : {ticket_data.shape[0]} \n")
ticket_data.head()

Instances de données disponibles : 74168 



,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00,2017-10-13 20:10:00+00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542


### Il y a t-il de données manquantes dans les dataset ?

Nous avons au total **7672** données manquantes dans le dataset **cities**.

In [9]:
cities.isnull().sum().sum()

7672

Il n'y a aucune donnée manquante dans le dataset **stations**

In [10]:
stations.isnull().sum().sum()

0

Nous observons 26 données manquantes dans le dataset **providers**

In [11]:
print("Total des données manquantes :", providers.isna().sum().sum(), "\n")
providers.isna().sum().sum()

Total des données manquantes : 26 



26

Nous avons au total **165764** données manquantes dans le dataset **ticket_data**

In [12]:
ticket_data.isna().sum().sum()

165764

<div style="background-color:#5bbbcf;padding:2px;border-radius:4px">  
    <h1 align="center" style="color:white;">1. Extraction des informations</h1>  
</div>

## 1. Prix min, max, moyen

Ici, nous nous intéressons aux prix minimum, maximum et moyen.

Le Dataset `ticket_data` contient toutes les informations nécessaire pour obtenir le résultat. 

In [13]:
min_price  = ticket_data.price_in_cents.min()   # Prix min de tout les trajets
max_price  = ticket_data.price_in_cents.max()   # Prix max de tout les trajets
mean_price = ticket_data.price_in_cents.mean()  # Prix moyenne de tout les trajets

print(f"Le prix minimum de tout les trajets est : {min_price}")
print(f"Le prix maximum de tout les trajets est : {max_price}")
print(f"Le prix moyen de tout les trajets est : {mean_price}")

Le prix minimum de tout les trajets est : 300
Le prix maximum de tout les trajets est : 38550
Le prix moyen de tout les trajets est : 4382.711061374178


In [14]:
# Prix min, max et moyen de chaque trajet
priceParTrajet = ticket_data.groupby(['o_city', 'd_city']).agg({'price_in_cents': ['min', 'max','mean']})
priceParTrajet.columns = ['price_min', 'price_max', 'price_mean']
priceParTrajet = priceParTrajet.reset_index()
priceParTrajet.head(3)

,o_city,d_city,price_min,price_max,price_mean
0,5,23,18600,22000,20320.000000
1,6,227,9860,13650,11755.000000
2,6,504,2000,8920,4042.666667


In [15]:
# Fonction de récupération du nom d'une ville à partir de son id
def get_city_name(id_city):
    return cities.query(f'id=={id_city}')['unique_name'].values[0]

priceParTrajet['o_city_name'] = priceParTrajet['o_city'].apply(get_city_name) # Récupérer le nom des villes de départ
priceParTrajet['d_city_name'] = priceParTrajet['d_city'].apply(get_city_name) # Récupérer le nom des villes d'arrivé

priceParTrajet[['price_min','price_max','price_mean','o_city_name','d_city_name']].head(10).style.background_gradient(cmap=cmr).set_precision(2)

,price_min,price_max,price_mean,o_city_name,d_city_name
0,18600,22000,20320.00,agde,amsterdam
1,9860,13650,11755.00,agen,dijon
2,2000,8920,4042.67,agen,marseille
3,2600,3190,2797.50,agen,paris
4,700,2420,864.63,agen,toulouse
5,3700,4050,3833.33,agen,marseille-aeroport
6,5090,5290,5190.00,aix-en-provence,angers
7,2000,3280,2630.00,aix-en-provence,annecy
8,2390,10740,6405.24,aix-en-provence,clermont-ferrand
9,1600,7100,4389.39,aix-en-provence,la-rochelle


## Conclusion

Comme nous pouvons le constater, le Dataframe **priceParTrajet** résume les prix min, max et moyen de chaque trajet. Par exemple  : 

- Les prix min, max et moyen du trajet `agde`-`amsterdam` sont respectivement **18600**, **22000** et **20320** cent ;
- Les prix min, max et moyen du trajet `agen`-`paris` sont respectivement **2600**, **3190** et  **2797.50** cent ;
- Les prix min, max et moyen du trajet `aix-en-provence`-`angers` sont respectivement **5090**, **5290** et  **5190** cent
- etc...

## 2. Durée min, max, moyenne par trajet

Ici, nous nous intéressons à la durée minimum, maximum et moyenne par trajet.

Le Dataset `ticket_data` contient toutes les informations nécessaire pour obtenir le résultat. 

**Vérification de la présence des valeurs manquantes**

Nous devons, avant tout, nous assurer qu'il n'y a pas de données manquantes dans les colonnes des trajets : `departure_ts` marquant le temps de départ et `arrival_ts` marquant le temps d'arrivé.

In [16]:
# Vérification de la présence des valeurs manquantes
TRAJET_COLONNES = ['departure_ts','arrival_ts']
ticket_data[TRAJET_COLONNES].isna().sum()

departure_ts    0
arrival_ts      0
dtype: int64

Nous remarquons donc qu'il n'y a aucune donnée manquante dans notre Dataset pour ces deux colonnes.

Par la suite, intéressons nous au type de donnée des deux colonnes.

In [17]:
# Affichage du type de données de chacune des deux colonnes
ticket_data[TRAJET_COLONNES].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74168 entries, 0 to 74167
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   departure_ts  74168 non-null  object
 1   arrival_ts    74168 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


Nous remarquons que le type des données est `Object`. Nous aurons donc besoin de les convertir en `datetime` pour pourvoir mieux les manipuler. Pour ce faire, nous allons utiliser l'attribut `to_datetime` de **Pandas**.

In [18]:
# Conversion des types de données en datetime
ticket_data['departure_ts'] = pd.to_datetime(ticket_data.departure_ts)
ticket_data['arrival_ts'] = pd.to_datetime(ticket_data.arrival_ts)

# Affichage de nouveau type de données de chacune des deux colonnes
ticket_data[TRAJET_COLONNES].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74168 entries, 0 to 74167
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   departure_ts  74168 non-null  datetime64[ns, UTC]
 1   arrival_ts    74168 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2)
memory usage: 1.1 MB


**Calcule de la durée de chaque trajet**

Nous pouvons à présent calculer la durée de chaque trajet, en faisant une soustraction de la durée de départ dans la durée d'arrivé.

In [19]:
ticket_data['duree'] = ticket_data['arrival_ts'] - ticket_data['departure_ts'] 
ticket_data[['departure_ts','arrival_ts','duree']].head(3)

,departure_ts,arrival_ts,duree
0,2017-10-13 14:00:00+00:00,2017-10-13 20:10:00+00:00,0 days 06:10:00
1,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,0 days 17:50:00
2,2017-10-13 13:27:00+00:00,2017-10-14 21:24:00+00:00,1 days 07:57:00


In [20]:
min_ts  = ticket_data.duree.min()   # Durée min de tout les trajets
max_ts  = ticket_data.duree.max()   # Durée max de tout les trajets
mean_ts = ticket_data.duree.mean()  # Durée moyenne de tout les trajets

print(f"La durée minimale de l'ensemble des trajets est : {min_ts}")
print(f"La durée maximale de l'ensemble des trajets est : {max_ts}")
print(f"La durée moyenne de l'ensemble des trajets est : {mean_ts}")

La durée minimale de l'ensemble des trajets est : 0 days 00:20:00
La durée maximale de l'ensemble des trajets est : 20 days 12:51:00
La durée moyenne de l'ensemble des trajets est : 0 days 07:04:37.247600043


#### Quel est le nombre total de trajet que nous avons ?

In [21]:
TOTAL_TRAJET = len(ticket_data.groupby('o_city')['d_city'].value_counts())
print(f"Nous avons au total {TOTAL_TRAJET} trajets.")

Nous avons au total 1437 trajets.


**Durée min, max, moyenne par trajet**

In [22]:
# Durée min, max, moyenne par trajet
dureeParTrajet = ticket_data.groupby(['o_city', 'd_city']).agg({'duree': ['min', 'max','mean']})
dureeParTrajet.columns = ['duree_min', 'duree_max', 'duree_mean']
dureeParTrajet = dureeParTrajet.reset_index()

dureeParTrajet['o_city_name'] = dureeParTrajet['o_city'].apply(get_city_name) # Récupérer le nom des villes de départ
dureeParTrajet['d_city_name'] = dureeParTrajet['d_city'].apply(get_city_name) # Récupérer le nom des villes d'arrivé
dureeParTrajet[['duree_min','duree_max','duree_mean','o_city_name','d_city_name']].head(10).style.background_gradient(cmap=cmr)

,duree_min,duree_max,duree_mean,o_city_name,d_city_name
0,0 days 08:53:00,0 days 15:54:00,0 days 10:18:48,agde,amsterdam
1,0 days 12:24:00,0 days 15:01:00,0 days 13:42:30,agen,dijon
2,0 days 05:36:00,0 days 12:20:00,0 days 08:17:24,agen,marseille
3,0 days 09:40:00,0 days 14:30:00,0 days 12:10:00,agen,paris
4,0 days 01:00:00,0 days 04:11:00,0 days 01:19:54.626865671,agen,toulouse
5,0 days 05:00:00,0 days 08:00:00,0 days 05:50:00,agen,marseille-aeroport
6,0 days 14:35:00,0 days 15:10:00,0 days 14:52:30,aix-en-provence,angers
7,0 days 21:45:00,1 days 02:45:00,1 days 00:09:00,aix-en-provence,annecy
8,0 days 04:06:00,1 days 07:40:00,0 days 11:26:25.714285714,aix-en-provence,clermont-ferrand
9,0 days 08:00:00,1 days 18:40:00,0 days 15:57:34.545454545,aix-en-provence,la-rochelle


In [23]:
#grouped_multiple['d_cit_name'] = grouped_multiple.merge(cities[['id','unique_name']], how="inner", left_on='d_city', right_on='id')['unique_name']
#grouped_multiple['o_cit_name'] = grouped_multiple.merge(cities[['id','unique_name']], how="inner", left_on='o_city', right_on='id')['unique_name']
#grouped_multiple.tail()

#grouped_multiple['o_city_name'] = pd.merge(left=grouped_multiple, right=cities, how="inner", left_on="o_city", right_on="id")['unique_name']
#grouped_multiple['d_city_name'] = pd.merge(left=grouped_multiple, right=cities,  how="right", left_on="d_city", right_on="id")['unique_name']
#grouped_multiple.tail()

## Conclusion

Comme nous pouvons le constater, le Dataframe **dureeParTrajet** résume les durées min, max et moyenne de chaque trajet. Par exemple  : 

- Les durées min, max et moyenne du trajet `agde`-`amsterdam` sont respectivement **08h53min**, **15h01min** et **10h18min48sec**
- Les durées min, max et moyenne du trajet `agen`-`marseille` sont respectivement **05h36min**, **12h20min** et  **08h17min24sec**
- Les durées min, max et moyenne du trajet `aix-en-provence`-`clermont-ferrand` sont respectivement **04h06min**, **1 jour 07h40min** et  **11h26min25sec**
- etc...

<div style="background-color:#5bbbcf;padding:2px;border-radius:4px">  
    <h2 align="center" style="color:white;">2. Différence de prix moyen et durée selon le train, le bus et le covoit selon la distance du trajet 
 </h2>  
</div>

#### Calcul de la distance de chaque trajet 

In [24]:
from math import sin, cos, sqrt, atan2, radians

distTrajets = ticket_data[['company','price_in_cents','o_city','d_city','duree']].copy()
distTrajets.head(3)

,company,price_in_cents,o_city,d_city,duree
0,8385,4550,611,542,0 days 06:10:00
1,9,1450,611,542,0 days 17:50:00
2,8377,7400,611,542,1 days 07:57:00


In [25]:
kms = []
R = 6378.0 # le rayon approximatif de la terre en km

for i in range(distTrajets.shape[0]):
    id_o_city = distTrajets['o_city'].iloc[i]
    id_d_city = distTrajets['d_city'].iloc[i]
    
    coordonnes = cities.query(f"id=={id_o_city}")[['latitude','longitude']].values[0]
    lat1 = radians(coordonnes[0])
    lon1 = radians(coordonnes[1])
    
    coordonnes = cities.query(f"id=={id_d_city}")[['latitude','longitude']].values[0]
    lat2 = radians(coordonnes[0])
    lon2 = radians(coordonnes[1])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    dist = R * c
    dist = np.round(dist, 3)
    
    kms.append(dist)
    
distTrajets['distance_km'] = kms
distTrajets.head(3)

,company,price_in_cents,o_city,d_city,duree,distance_km
0,8385,4550,611,542,0 days 06:10:00,503.75
1,9,1450,611,542,0 days 17:50:00,503.75
2,8377,7400,611,542,1 days 07:57:00,503.75


#### Récupération du type de transport de chaque trajet

In [26]:
# Type de moyen de transport proposer par la compagnie 9
providers.query("company_id==9")['transport_type'].values[0]

'bus'

Malheureusement, pour certaines compagnies comme la compagnie `8385`, nous ne disposont pas des informations sur le type de transport.

In [27]:
# Type de moyen de transport proposer par la compagnie 8385
comp = 8385 # id de la compagnie
try:
    print(providers.query(f"company_id=={comp}")['transport_type'].values[0])
except:
    print(f"Pas de moyen de transport proposer par la compagnie {comp}")

Pas de moyen de transport proposer par la compagnie 8385


In [28]:
# Fonction de récupération du type de transport proposé par une compagnie
# On retourne NaN si le type n'est pas fourni
def get_transport_type(id_transport_type):
    try:
        return providers.query(f"company_id=={id_transport_type}")['transport_type'].values[0]
    except:
        return np.NaN

# Récupération du type de transport de chaque trajet
distTrajets['transport_type'] = distTrajets['company'].apply(get_transport_type)
distTrajets.head(3)

,company,price_in_cents,o_city,d_city,duree,distance_km,transport_type
0,8385,4550,611,542,0 days 06:10:00,503.75,NaN
1,9,1450,611,542,0 days 17:50:00,503.75,bus
2,8377,7400,611,542,1 days 07:57:00,503.75,NaN


In [29]:
val = distTrajets.shape[0] - distTrajets['transport_type'].isna().sum()
print(f"Nous avons {val} données utilisables sur les {distTrajets.shape[0]} données fournies")

Nous avons 3574 données utilisables sur les 74168 données fournies


Nous allons donc continuer les analyses avec les **3574** données utilisables.

In [30]:
# Récupération des 3574 données dans un nouveau dataframe (df) 
df = distTrajets.dropna(axis=0)

# Affichage du nombre de type de transport dont nous disposons
df.transport_type.value_counts()

bus           3560
train            9
carpooling       5
Name: transport_type, dtype: int64

In [31]:
print(f"La distance minimale en km de l'ensemble des trajets est {df.distance_km.min()}")
print(f"La distance maximale en km de l'ensemble des trajets est {df.distance_km.max()}")
print(f"La distance moyenne en km de l'ensemble des trajets est {df.distance_km.mean()}")

La distance minimale en km de l'ensemble des trajets est 30.547
La distance maximale en km de l'ensemble des trajets est 1757.986
La distance moyenne en km de l'ensemble des trajets est 452.46020397313936


In [32]:
# Fonction pour récupérer les plages de distance en km
def get_distance_range(km):
    if km <= 250:
        return '0-250km'
    elif km <= 500:
        return '251-500km'
    elif km <= 1000:
        return '501-1000km'
    else:
        return '1001+km'
    
warnings.filterwarnings('ignore')
df['distance_range'] = df['distance_km'].apply(get_distance_range)
df.head()

,company,price_in_cents,o_city,d_city,duree,distance_km,transport_type,distance_range
1,9,1450,611,542,0 days 17:50:00,503.750,bus,501-1000km
51,9,1990,628,453,0 days 06:35:00,206.069,bus,0-250km
53,9,1800,628,453,1 days 06:50:00,206.069,bus,0-250km
55,9,1590,628,453,0 days 04:10:00,206.069,bus,0-250km
57,9,1100,628,453,1 days 01:05:00,206.069,bus,0-250km


In [33]:
# Agrégation des prix et durées selon la distance et moyen de transport
df1 = df.groupby(['distance_range', 'transport_type']).agg({'duree': 'mean', 'price_in_cents': 'mean'})
df1.style.background_gradient(cmap=cmr)

In [34]:
# Récupération des données des prix de chaque type de transport
df_prix_bus = df1.query("transport_type=='bus'").groupby(['distance_range']).agg({'price_in_cents': 'mean'})
df_prix_train = df1.query("transport_type=='train'").groupby(['distance_range']).agg({'price_in_cents': 'mean'})
df_prix_car = df1.query("transport_type=='carpooling'").groupby(['distance_range']).agg({'price_in_cents': 'mean'})

# Visualisation Interactive des données
fig = go.Figure(data=[
    go.Bar(name='bus', x=df_prix_bus.index, y=df_prix_bus.price_in_cents, marker_color='#1a8df6'),
    go.Bar(name='train', x=df_prix_train.index, y=df_prix_train.price_in_cents, marker_color='#dc3545'),
    go.Bar(name='carpooling', x=df_prix_car.index, y=df_prix_car.price_in_cents, marker_color='orange'),
])    

fig.update_layout(barmode='group',title='Rapport Prix moyen/distance des types de transport')
fig.show()

In [35]:
# Récupération des données des durées de chaque type de transport
df_duree_bus = df1.query("transport_type=='bus'").groupby(['distance_range']).agg({'duree': 'mean'})
df_duree_train = df1.query("transport_type=='train'").groupby(['distance_range']).agg({'duree': 'mean'})
df_duree_car = df1.query("transport_type=='carpooling'").groupby(['distance_range']).agg({'duree': 'mean'})

# Visualisation Interactive des données
fig = go.Figure(data=[
    go.Bar(name='bus', x=df_duree_bus.index, y=df_duree_bus.duree, marker_color='#1a8df6'),
    go.Bar(name='train', x=df_duree_train.index, y=df_duree_train.duree, marker_color='#dc3545'),
    go.Bar(name='carpooling', x=df_duree_car.index, y=df_duree_car.duree, marker_color='orange'),
])    

fig.update_layout(barmode='group',title='Rapport Durée moyenne/Distance des types de transport')
fig.show()

## Conclusion

Comme nous pouvons le remarquer, le Dataframe **df1** résume ...

- Sur une distance de **0-250 km** nous n'avons deux types de transport : 
    - le bus avec une durée moyenne de `10h 43min 36sec` et un prix moyen de `2031.41` cent
    - le carpooling (covoiturage) avec une durée moyenne de `14h 15min` et un prix moyen de `1990` cent
    
    
- Sur une distance **251-500 km** nous avons trois types de transport : 
    - le covoiturage avec une durée moyenne de `09h 40min` et un prix moyen de `3945` cent
    - le train avec une durée moyenne de `09h 33min 20sec` et un prix moyen de `3100` cent
    - le bus avec une durée moyenne de `17h 39min 51sec` et un prix moyen de `4151.41` cent
    
    
- Sur une distance **501-1000 km** nous n'avons trois types de transport : 
    - le covoiturage avec une durée moyenne de `11h 32min 30sec` et un prix moyen de `4732` cent
    - le train avec une durée moyenne de `19h00` et un prix moyen de `3596.66` cent
    
    
 - Sur une distance de plus de **1001 km** nous n'avons qu'un seul type de transport : 
    - le bus avec une durée moyenne de `1jour 06h 42min 33sec` et un prix moyen de `8038.97` cent
  
  
  
  
## Remarques

Il est tout à fait normal de reporter les observations suivants :
      
- La plus **petite durée moyenne** fait en déplacement est `09h 33min 20sec`. Elle se fait en **train** sur une distance de **251-500 km**;

- La plus **longue durée moyenne** fait en déplacement est `1jour 06h 42min 33sec`. Elle se en **bus** sur une distance de **1001+ km**;

- Le plus **petit prix moyen** payé en déplacement est `1990` cent, en **covoiturage** et sur une distance de **0-250 km**;

- Le plus **grand prix moyen** payé en déplacement est `8038.97` cent, dans un **bus** et sur une distance de **plus de 1001 km**.


<div style="background-color:#5bbbcf;padding:2px;border-radius:4px">  
    <h2 align="center" style="color:white;">3. Bonus </h2>  
</div>

**Remplacer quelques valeurs manquantes du dataset providers**

Nous avons précédemment remarquer que des valeurs manques dans les colonnes : `provider_id`, `has_wifi`, `has_plug`, `has_adjustable_seats` et `has_bicycle` du dataset **providers**. 

Dans cette section, nous allons nous intéresser à remplacer les valeurs manquantes des colonnes `has_wifi`, `has_plug`, `has_adjustable_seats` et `has_bicycle`.

In [36]:
# Les colonnes des valeurs manquantes
MISSING_COL = [col for col in providers.columns.to_list() if providers[col].isna().any()]
providers[MISSING_COL].isna().sum()

provider_id             14
has_wifi                 3
has_plug                 3
has_adjustable_seats     3
has_bicycle              3
dtype: int64

Nous remarquons l'absence de trois valeurs manquantes dans chacune de nos quatres colonnes cibles.

Intéressons nous aux instances de ces valeurs manquantes.

In [37]:
# Les instances pour lesquelles des valeurs manques
providers[providers.has_plug.isna()]

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
212,8374,20,NaN,transdev,Transdev,NaN,NaN,NaN,NaN,bus
224,8387,30,NaN,vatry,Vatry,NaN,NaN,NaN,NaN,bus
225,8388,40,NaN,beauval,Beauval,NaN,NaN,NaN,NaN,bus


Nous observons que :
- Seulement trois différentes instances sont concernées par les valeurs manquantes de nos colonnes cibles;
- Et que trois différentes compagnies sont impliquées sur ces instances;


Nous proposons de nous intéresser aux offres habituelles de ces 3 compagnies sur chacune de nos colonnes cibles.

### 1. Compagnie **20**

In [38]:
# Les offres de la compagnie 20
providers.query("company_id==20")

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
209,8371,20,I,isilines,Isilines,True,True,True,False,bus
210,8372,20,E,eurolines,Eurolines,False,False,True,False,bus
211,8373,20,M,movelia,Movelia,False,False,False,False,bus
212,8374,20,NaN,transdev,Transdev,NaN,NaN,NaN,NaN,bus


En regardant de près les offres de la compagnie 20, nous nous rendons compte que :

- la plupart des valeurs des colonnes `has_wifi`, `has_plug` et `has_bicycle` contient la valeur *False*. Pour ces colonnes, nous allons donc remplacer par *False*, la valeur manquante de l'instance de la compagnie 20.
- la plupart des valeurs de la colonne `has_adjustable_seats` contient la valeur *True*. Pour cette colonne, nous allons donc remplacer par *True*, la valeur manquante de l'instance de la compagnie 20.

In [39]:
# Remplacement des valeurs manquantes des colonnes "has_plug", "has_wifi" et "has_bicycle"
for col in ['has_plug','has_wifi','has_bicycle']:
    providers[col][providers.id==8374] = False
    
# Remplacement de la valeur manquante de colonne "has_adjustable_seats"
providers['has_adjustable_seats'][providers.id==8374] = True

providers.query("company_id==20")

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
209,8371,20,I,isilines,Isilines,True,True,True,False,bus
210,8372,20,E,eurolines,Eurolines,False,False,True,False,bus
211,8373,20,M,movelia,Movelia,False,False,False,False,bus
212,8374,20,NaN,transdev,Transdev,False,False,True,False,bus


### 2. Compagnies **30** & **40**


In [40]:
# Les offres des compagnies 30 & 40
providers.query("company_id in [30, 40]")

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
224,8387,30,NaN,vatry,Vatry,NaN,NaN,NaN,NaN,bus
225,8388,40,NaN,beauval,Beauval,NaN,NaN,NaN,NaN,bus


Aucune des compagnies **30** et **40** ne possède d'autres offres. Nous ne pouvons remplacer les valeurs manquantes de nos colonnes cibles pour ces 2 compagnies avec la méthode précédente. Alors, comment procéder ?

Remarquons que le moyen de transport que proposent nos 2 compagnies est le **bus**. Une idée pourrait être de remplacer les valeurs manquantes par les valeurs les plus fréquentes de nos colonnes cibles, pour le type de transport **bus**.

In [41]:
MISSING_COLONNES = ['has_wifi','has_plug','has_adjustable_seats','has_bicycle'] # Nos colonnes cibles
providers_bus = providers.query("transport_type=='bus'") # Les données ayant pour type de transport "bus"

# les valeurs fréqentes dans chaque colonnes
for col in MISSING_COLONNES:
    print(providers_bus[col].value_counts())

False    209
True       3
Name: has_wifi, dtype: int64
False    210
True       2
Name: has_plug, dtype: int64
False    207
True       5
Name: has_adjustable_seats, dtype: int64
False    212
Name: has_bicycle, dtype: int64


Nous observons que, pour chacune de nos 4 colonnes, la valeur la plus fréquente est *False*. Nous pouvons ainsi remplacer les valeurs manquantes par *False*, dans chacune de nos 4 colonnes cibles.

In [42]:
# Remplacement des valeurs null par la valeur "False", dans chacune des 4 colonnes
for col in MISSING_COLONNES:
    providers[col] = providers[col].fillna(False)

# Aucune de nos 4 colonnes ne contient de valeur null
providers[MISSING_COLONNES].isna().any()

has_wifi                False
has_plug                False
has_adjustable_seats    False
has_bicycle             False
dtype: bool

# Ressources

> [Calcul de durée par trajet](https://jamesrledoux.com/code/group-by-aggregate-pandas)

> [Calcul de distance entre deux points](https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude)